In [6]:
# Libreria para gestionar archivos
import os
# Libreria para trabajar con redes neuronales convolucionales
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras.layers import Convolution2D, MaxPooling2D
from tensorflow.keras import backend as K
# Libreria para trabajar con imagenes
import numpy as np
import matplotlib.pyplot as plt
import re
%matplotlib inline
from sklearn.model_selection import train_test_split

In [7]:
K.clear_session()
dirname = os.path.join(os.getcwd(), 'Data')
imgpath = dirname + os.sep 
 
images = []
directories = []
dircount = []
prevRoot=''
cant=0
 
print("Leyendo imagenes de ",imgpath)
 
for root, dirnames, filenames in os.walk(imgpath):
    for filename in filenames:
        if re.search("\.(jpg|jpeg|png|bmp|tiff)$", filename):
            cant=cant+1
            filepath = os.path.join(root, filename)
            image = plt.imread(filepath)
            images.append(image)
            b = "Leyendo..." + str(cant)
            print (b, end="\r")
            if prevRoot !=root:
                print(root, cant)
                prevRoot=root
                directories.append(root)
                dircount.append(cant)
                cant=0
dircount.append(cant)
 
dircount = dircount[1:]
dircount[0]=dircount[0]+1
print('Directorios leidos:',len(directories))
print("Imagenes en cada directorio", dircount)
print('Suma Total de imagenes en subdirs:',sum(dircount))

Leyendo imagenes de  c:\Users\yjot2\Downloads\RNC\Data\
c:\Users\yjot2\Downloads\RNC\Data\amarillo 1
c:\Users\yjot2\Downloads\RNC\Data\blanco 1000
c:\Users\yjot2\Downloads\RNC\Data\gris 1000
c:\Users\yjot2\Downloads\RNC\Data\marron 1000
c:\Users\yjot2\Downloads\RNC\Data\rojo 1000
Directorios leidos: 5
Imagenes en cada directorio [1001, 1000, 1000, 1000, 999]
Suma Total de imagenes en subdirs: 5000


In [8]:
labels=[]
indice=0
for cantidad in dircount:
    for i in range(cantidad):
        labels.append(indice)
    indice=indice+1
print("Cantidad etiquetas creadas: ",len(labels))
 
deportes=[]
indice=0
archivoTxt = open(".\Model\labels.txt", "w")
for directorio in directories:
    name = directorio.split(os.sep)
    print(indice , name[len(name)-1])
    deportes.append(name[len(name)-1])
    # Crear archivo TXT de etiquetas
    archivoTxt.write(name[len(name)-1])
    archivoTxt.write("\n")
    indice=indice+1
archivoTxt.close()
 
y = np.array(labels)
X = np.array(images, dtype=np.uint8) # convierto de lista a numpy
 
# Find the unique numbers from the train labels
clases = np.unique(y)
nClases = len(clases)
print('Total number of outputs : ', nClases)
print('Output classes : ', clases)

Cantidad etiquetas creadas:  5000
0 amarillo
1 blanco
2 gris
3 marron
4 rojo
Total number of outputs :  5
Output classes :  [0 1 2 3 4]


In [9]:
# Mezclar todo y crear los grupos de entrenamiento y testing
porcentajeTest = 0.3
train_X,test_X,train_Y,test_Y = train_test_split(X,y,test_size=porcentajeTest)
print('Training data shape : ', train_X.shape, train_Y.shape)
print('Testing data shape : ', test_X.shape, test_Y.shape)
 
train_X = train_X.astype('float32')
test_X = test_X.astype('float32')
train_X = train_X / 255.
test_X = test_X / 255.
 
# Change the labels from categorical to one-hot encoding
train_Y_one_hot = to_categorical(train_Y)
test_Y_one_hot = to_categorical(test_Y)
 
# Display the change for category label using one-hot encoding
print('Original label:', train_Y[0])
print('After conversion to one-hot:', train_Y_one_hot[0])
 
train_X,valid_X,train_label,valid_label = train_test_split(train_X, train_Y_one_hot, test_size=porcentajeTest, random_state=13)
 
print(train_X.shape,valid_X.shape,train_label.shape,valid_label.shape)

Training data shape :  (3500, 200, 200, 3) (3500,)
Testing data shape :  (1500, 200, 200, 3) (1500,)
Original label: 1
After conversion to one-hot: [0. 1. 0. 0. 0.]
(2450, 200, 200, 3) (1050, 200, 200, 3) (2450, 5) (1050, 5)


In [13]:
# Parametros
epocas = 10 # Numero de iteraciones para aprender y predecir la data 
longitud, altura = 200, 200 # Tamano de imagenes a procesar en px
batch_size = 128 # Ran go de imagenes a analizar por cada paso (entrenamiento y validacion)
tamano_pool = (2, 2) # Tamano de matriz de caracteristicas especificas
lr = 0.0001 # Tasa de aprendizaje

cnn = Sequential()
# Extraccion de caracteristicas generales y especificas #1 
cnn.add(Convolution2D(32, (3, 3), padding="same", input_shape=(longitud, altura, 3)))
cnn.add(MaxPooling2D(pool_size=tamano_pool))
# Extraccion de caracteristicas generales y especificas #2
cnn.add(Convolution2D(64, (3, 3), padding="same"))
cnn.add(MaxPooling2D(pool_size=tamano_pool))
# Extraccion de caracteristicas generales y especificas #3
cnn.add(Convolution2D(128, (3, 3), padding="same"))
cnn.add(MaxPooling2D(pool_size=tamano_pool))
# Extraccion de caracteristicas generales y especificas #4
cnn.add(Convolution2D(256, (3, 3), padding="same"))
cnn.add(MaxPooling2D(pool_size=tamano_pool))
# Extraccion de caracteristicas generales y especificas #5
cnn.add(Convolution2D(512, (3, 3), padding="same"))
cnn.add(MaxPooling2D(pool_size=tamano_pool))
# Extraccion de caracteristicas generales y especificas #6
cnn.add(Convolution2D(1024, (2, 2), padding="same"))
cnn.add(MaxPooling2D(pool_size=tamano_pool))
# Aplanado de imagenes, capa full conectada y aprendizaje por vias aleatorias
cnn.add(Flatten())
cnn.add(Dense(100, activation='relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(100, activation='relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(100, activation='relu'))
cnn.add(Dropout(0.5))
# Capa full conectada final
cnn.add(Dense(nClases, activation='softmax'))

cnn.summary() # Ver resumen de la estructura de capas

# Indicaciones de perdida, ganancia, optimizacion y entrenamiento del modelo
cnn.compile(loss='categorical_crossentropy',
            optimizer=optimizers.Adam(learning_rate=lr),
            metrics=['accuracy'])

salida = cnn.fit(train_X, train_label, batch_size=batch_size,epochs=epocas,verbose=1,validation_data=(valid_X, valid_label))

#Guardado del modelo keras
target_dir = os.path.join(os.getcwd(), 'Model')
if not os.path.exists(target_dir):
  os.mkdir(target_dir)
cnn.save(target_dir + '\modelo.h5')
#Conversion a un modelo TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(cnn)
tflite_model = converter.convert()
with open('.\Model\model.tflite', 'wb') as f:
  f.write(tflite_model)
print('Modelo convertido a tflite exitosamente')

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (None, 200, 200, 32)      896       
                                                                 
 max_pooling2d_18 (MaxPoolin  (None, 100, 100, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_19 (Conv2D)          (None, 100, 100, 64)      18496     
                                                                 
 max_pooling2d_19 (MaxPoolin  (None, 50, 50, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_20 (Conv2D)          (None, 50, 50, 128)       73856     
                                                                 
 max_pooling2d_20 (MaxPoolin  (None, 25, 25, 128)     

INFO:tensorflow:Assets written to: C:\Users\yjot2\AppData\Local\Temp\tmp9krj36mj\assets


Modelo convertido a tflite exitosamente
